In [1]:
import os
import shutil
import random

# Set the paths to your dataset and the train/test folders
data_dir = "/kaggle/input/covid-19/Covid19"
train_dir = "/kaggle/working/train"
test_dir = "/kaggle/working/test"

# Create the train and test folders if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Create subfolders for the classes (Normal and Covid-19)
os.makedirs(os.path.join(train_dir, "Normal"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "COVID"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "Normal"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "COVID"), exist_ok=True)

# Set the train/test split ratio (e.g., 0.8 for 80% train, 0.2 for 20% test)
train_ratio = 0.8

# Loop through the classes
for class_name in ["Normal", "COVID"]:
    class_dir = os.path.join(data_dir, class_name)
    files = os.listdir(class_dir)
    
    # Shuffle the files randomly
    random.shuffle(files)
    
    # Calculate the number of files for train and test sets
    num_train = int(len(files) * train_ratio)
    
    # Copy the files to the train and test folders
    for i, file in enumerate(files):
        src = os.path.join(class_dir, file)
        if i < num_train:
            dst = os.path.join(train_dir, class_name, file)
        else:
            dst = os.path.join(test_dir, class_name, file)
        shutil.copy(src, dst)

In [2]:
%pip install -q flwr[simulation] 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.3 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
cudf 23.8.0 requires cuda

In [3]:
# Install PyTorch with GPU support
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # Required for Kaggle to work with PyTorch
import torch
torch.cuda.is_available()  # Check if GPU is available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__}"
)

Training on cuda using PyTorch 2.1.2


In [1]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import flwr as fl
import matplotlib.pyplot as plt
import ipywidgets as widgets
import IPython.display as display_output
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
from collections import OrderedDict
from typing import List, Tuple
from IPython.display import display, clear_output
from IPython.display import display, HTML, clear_output
from flwr.common import Metrics

global num_c, trainloaders, valloaders, testloader, model, classes
num_clients=15
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Dataset paths
dataset_paths = '/kaggle/working/'

datatrain = f'{dataset_paths}/train'
datatest = f'{dataset_paths}/test'

BATCH_SIZE = 32
img_size = (224, 224)

# Download and transform
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = ImageFolder(datatrain, transform=transform)
testset = ImageFolder(datatest, transform=transform)
size_label = widgets.Label()
size_label.value = f'The size of data: {len(trainset) + len(testset)}'
print(size_label)

# Get the classes from the trainset
classes = trainset.classes
num_c = len(classes)

# Split training set into num_clients partitions to simulate the individual dataset
partition_size = len(trainset) // num_clients
remainder = len(trainset) % num_clients
lengths = [partition_size + remainder] + [partition_size] * (num_clients - 1)
datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

# Split each partition into train/val and create DataLoader
trainloaders = []
valloaders = []
for ds in datasets:
    len_val = len(ds) // 10  # 10 % validation set
    len_train = len(ds) - len_val
    lengths = [len_train, len_val]
    ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
    trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
    print("hi")
    valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    

def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)
    
class Net(nn.Module):
    def __init__(self, num_c):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 28 * 28, 264)
        self.fc2 = nn.Linear(264,num_c)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        x = self.pool3(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x


# Function to train the network
def train(net, trainloader, valloader, epochs: int,patience: int = 5, verbose=True):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    best_val_acc = 0.0
    early_stop_counter = 0
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        val_loss, val_acc = test(net, valloader)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f"Early stopping after {epoch + 1} epochs.")
                break

        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}, val loss {val_loss}, val accuracy {val_acc}")

# Function to test the network
def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

def average_FedAvg(metrics: List[Tuple[int, dict]]) -> dict:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

def fit_metrics_aggregation_fn(fit_metrics):
    # Aggregate and return custom metric (e.g., weighted average)
    accuracies = [num_examples * m["accuracy"] for num_examples, m in fit_metrics]
    examples = [num_examples for num_examples, _ in fit_metrics]
    return {"accuracy": sum(accuracies) / sum(examples)}

class FlowerNumPyClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)
    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader,self.valloader, epochs=1)
        
        # Get the accuracy on the training set
        correct, total = 0, 0
        self.net.eval()
        with torch.no_grad():
            for images, labels in self.trainloader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = self.net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        
        return get_parameters(self.net), len(self.trainloader), {"accuracy": float(accuracy)}
    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}
    
    
def client_fn(cid) -> FlowerNumPyClient:
    net = Net(num_c).to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    flower_numpy_client = FlowerNumPyClient(cid, net, trainloader, valloader)
    return flower_numpy_client.to_client()



def run_simulation(num_round,strategy_dropdown):   
    if strategy_dropdown == "FedAvg":
        strategy = fl.server.strategy.FedAvg(
            fraction_fit=1.0,
            fraction_evaluate=0.5,
            min_fit_clients=num_clients - 1,
            min_evaluate_clients=num_clients - 2,
            min_available_clients=num_clients,
            initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net(num_c))),
            evaluate_metrics_aggregation_fn=average_FedAvg,# <-- pass the metric aggregation function
            fit_metrics_aggregation_fn=fit_metrics_aggregation_fn,
        )

    # Update with your specific simulation code
    fl.common.logger.configure(identifier="myFlowerExperiment", filename="log.txt")
    output = fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=num_clients,
        config=fl.server.ServerConfig(num_rounds=num_round),
        strategy=strategy,
        client_resources={"num_cpus": 1, "num_gpus": 1},
    )

run_simulation(15,"FedAvg")

2024-05-06 22:19:33.362677: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 22:19:33.362735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 22:19:33.364167: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 22:19:36,609	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `

Label(value='The size of data: 13808')
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi


INFO :      Starting Flower simulation, config: num_rounds=15, no round_timeout
2024-05-06 22:19:39,885	INFO worker.py:1621 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'GPU': 2.0, 'object_store_memory': 8471637196.0, 'memory': 16943274395.0, 'CPU': 4.0, 'node:__internal_head__': 1.0, 'node:172.19.2.2': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Evaluating initial global parameters
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 15 clients (out of 15)
(pid=1493) 2024-05-06 22:19:44.154114: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN f

(ClientAppActor pid=1493) [Client 7] fit, config: {}
(ClientAppActor pid=1491) Epoch 1: train loss 0.02283284440636635, accuracy 0.6681749622926093, val loss 0.024708531490743976, val accuracy 0.6575342465753424
(ClientAppActor pid=1491) [Client 8] fit, config: {} [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(ClientAppActor pid=1491) Epoch 1: train loss 0.024059945717453957, accuracy 0.6606334841628959, val loss 0.017462497296398632, val accuracy 0.8082191780821918 [repeated 2x across cluster]
(ClientAppActor pid=1491) [Client 9] fit, config: {} [repeated 2x across cluster]
(ClientAppActor pid=1491) Epoch 1: train loss 0.021585525944828987, accuracy 0.7405731523378583, val loss 0.026954583925743625, val accuracy 0.7671232876712328 [repeated 2x across cluster]
(ClientAppActor pid=1491) [Client 6] fit, config: {

INFO :      aggregate_fit: received 15 results and 0 failures
INFO :      configure_evaluate: strategy sampled 13 clients (out of 15)


...

INFO :      aggregate_evaluate: received 13 results and 0 failures
INFO :      
INFO :      [ROUND 15]
INFO :      configure_fit: strategy sampled 15 clients (out of 15)


(ClientAppActor pid=1491) [Client 1] fit, config: {}
(ClientAppActor pid=1491) Epoch 1: train loss 0.007365334779024124, accuracy 0.9110105580693816, val loss 0.008376057629715907, val accuracy 0.9041095890410958
(ClientAppActor pid=1491) [Client 9] fit, config: {} [repeated 2x across cluster]
(ClientAppActor pid=1491) Epoch 1: train loss 0.00838552974164486, accuracy 0.8944193061840121, val loss 0.018541009458777024, val accuracy 0.863013698630137 [repeated 2x across cluster]
(ClientAppActor pid=1491) [Client 14] fit, config: {} [repeated 2x across cluster]
(ClientAppActor pid=1491) Epoch 1: train loss 0.009450645186007023, accuracy 0.8959276018099548, val loss 0.007859722086011546, val accuracy 0.9178082191780822 [repeated 2x across cluster]
(ClientAppActor pid=1491) [Client 3] fit, config: {} [repeated 2x across cluster]
(ClientAppActor pid=1491) Epoch 1: train loss 0.009572239592671394, accuracy 0.8748114630467572, val loss 0.008229240775108337, val accuracy 0.8904109589041096 [rep

INFO :      aggregate_fit: received 15 results and 0 failures
INFO :      configure_evaluate: strategy sampled 13 clients (out of 15)


(ClientAppActor pid=1491) [Client 0] evaluate, config: {}
(ClientAppActor pid=1491) [Client 8] evaluate, config: {} [repeated 12x across cluster]


INFO :      aggregate_evaluate: received 13 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 15 rounds in 985.60s
INFO :      History (loss, distributed):
INFO :      	('\tround 1: 0.027325331877036573\n'
INFO :      	 '\tround 2: 0.019714154621569075\n'
INFO :      	 '\tround 3: 0.018347301448236524\n'
INFO :      	 '\tround 4: 0.01785229515476649\n'
INFO :      	 '\tround 5: 0.017084143316365853\n'
INFO :      	 '\tround 6: 0.016492312260406445\n'
INFO :      	 '\tround 7: 0.0163678990998624\n'
INFO :      	 '\tround 8: 0.014774142141761518\n'
INFO :      	 '\tround 9: 0.013892678490082412\n'
INFO :      	 '\tround 10: 0.01173417277429806\n'
INFO :      	 '\tround 11: 0.01121600631009007\n'
INFO :      	 '\tround 12: 0.010301802920380378\n'
INFO :      	 '\tround 13: 0.010357042974386743\n'
INFO :      	 '\tround 14: 0.010399838890462868\n'
INFO :      	 '\tround 15: 0.009609000890113673\n')History (metrics, distributed, fit):
INFO :      	{'accuracy